In [1]:
import requests
from pathlib import Path

url = "https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv"

bronze_path = Path("/lakehouse/default/Files/bronze/taxi")
bronze_path.mkdir(parents=True, exist_ok=True)

file_path = bronze_path / "taxi_zone_lookup.csv"

response = requests.get(url)
response.raise_for_status()

with open(file_path, "wb") as f:
    f.write(response.content)

file_path



StatementMeta(, 8317ef89-ccf6-4a51-8b39-e75d6df452c9, 3, Finished, Available, Finished)

PosixPath('/lakehouse/default/Files/bronze/taxi/taxi_zone_lookup.csv')

In [2]:
from pyspark.sql import functions as F

df_zone_raw = (
    spark.read
    .option("header", True)
    .csv("Files/bronze/taxi/taxi_zone_lookup.csv")
)

df_zone_raw.show(5)
df_zone_raw.printSchema()


StatementMeta(, 8317ef89-ccf6-4a51-8b39-e75d6df452c9, 4, Finished, Available, Finished)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows

root
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [3]:
df_dim_zone = (
    df_zone_raw
    .select(
        F.col("LocationID").cast("int").alias("zone_id"),
        F.col("Borough").alias("borough"),
        F.col("Zone").alias("zone_name"),
        F.col("service_zone")
    )
    .dropDuplicates(["zone_id"])
)


StatementMeta(, 8317ef89-ccf6-4a51-8b39-e75d6df452c9, 5, Finished, Available, Finished)

In [4]:
df_dim_zone.count()
df_dim_zone.show(5, truncate=False)


StatementMeta(, 8317ef89-ccf6-4a51-8b39-e75d6df452c9, 6, Finished, Available, Finished)

+-------+-------------+-----------------------+------------+
|zone_id|borough      |zone_name              |service_zone|
+-------+-------------+-----------------------+------------+
|1      |EWR          |Newark Airport         |EWR         |
|2      |Queens       |Jamaica Bay            |Boro Zone   |
|3      |Bronx        |Allerton/Pelham Gardens|Boro Zone   |
|4      |Manhattan    |Alphabet City          |Yellow Zone |
|5      |Staten Island|Arden Heights          |Boro Zone   |
+-------+-------------+-----------------------+------------+
only showing top 5 rows



In [7]:
dim_zone_norm = (
    df_dim_zone
    .withColumn(
        "zone_name_norm",
        F.lower(F.trim(F.col("zone_name")))
    )
    .select("zone_id", "zone_name_norm", "zone_name", "borough")
)
dim_zone_norm.show(5, truncate=False)

StatementMeta(, 8317ef89-ccf6-4a51-8b39-e75d6df452c9, 9, Finished, Available, Finished)

+-------+-----------------------+-----------------------+-------------+
|zone_id|zone_name_norm         |zone_name              |borough      |
+-------+-----------------------+-----------------------+-------------+
|1      |newark airport         |Newark Airport         |EWR          |
|2      |jamaica bay            |Jamaica Bay            |Queens       |
|3      |allerton/pelham gardens|Allerton/Pelham Gardens|Bronx        |
|4      |alphabet city          |Alphabet City          |Manhattan    |
|5      |arden heights          |Arden Heights          |Staten Island|
+-------+-----------------------+-----------------------+-------------+
only showing top 5 rows



In [10]:
df_locations_silver = spark.read.table("df_locations_silver")

StatementMeta(, 8317ef89-ccf6-4a51-8b39-e75d6df452c9, 12, Finished, Available, Finished)

In [11]:
air_zone_map = (
    df_locations_silver
    .join(
        dim_zone_norm,
        df_locations_silver.location_name_norm == dim_zone_norm.zone_name_norm,
        "left"
    )
)


StatementMeta(, 8317ef89-ccf6-4a51-8b39-e75d6df452c9, 13, Finished, Available, Finished)

In [12]:
air_zone_map.select(
    "location_id",
    "name",
    "zone_name",
    "zone_id"
).show(50, truncate=False)


StatementMeta(, 8317ef89-ccf6-4a51-8b39-e75d6df452c9, 14, Finished, Available, Finished)

+-----------+--------------------------------------------------+-------------+-------+
|location_id|name                                              |zone_name    |zone_id|
+-----------+--------------------------------------------------+-------------+-------+
|384        |CCNY                                              |NULL         |NULL   |
|386        |Susan Wagner                                      |NULL         |NULL   |
|625        |Manhattan/IS143                                   |NULL         |NULL   |
|626        |Bronx - IS52                                      |NULL         |NULL   |
|628        |Maspeth                                           |Maspeth      |157    |
|631        |Queens                                            |NULL         |NULL   |
|642        |PS 19                                             |NULL         |NULL   |
|648        |Bklyn - PS 314                                    |NULL         |NULL   |
|662        |Division Street               

In [5]:
(
    df_dim_zone
    .write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("dbo.dim_zone")
)


StatementMeta(, 6b0bbe09-9573-4bdd-b074-1e3e4abb6f6a, 7, Finished, Available, Finished)

In [1]:
spark.sql("""
SELECT
    COUNT(*) AS trips,
    COUNT(zone_id) AS zones
FROM dbo.silver_taxi_trips t
LEFT JOIN dbo.dim_zone z
    ON t.PULocationID = z.zone_id;
""").show()


StatementMeta(, 6cdbbf21-4a5e-4814-a9c6-a410a04c6139, 3, Finished, Available, Finished)

+-------+-------+
|  trips|  zones|
+-------+-------+
|2840785|2840785|
+-------+-------+

